In [1]:
import sys
sys.path.append("../../")

from itops.storage.azure_blob.azure_blob_helper import AzureBlobHelper
from itops.config.configs import CONFIGS

True


In [2]:
description_column_name = "Text"
MODEL_NAME = "all-MiniLM-L6-v2"
CATEGORY = "ITSM-PARQUET2"
run_name = "RUN0006"
user_input = "Please extract the theme of the provided context in Maximum 5 words"
NUM_CLUSTERS = 6

In [3]:
from itops.runmanager.runmamanger import RunManager

c:\Ambarish\itops\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
run_manager = RunManager(description_column_name=description_column_name,
                         embedding_model_name=MODEL_NAME,
                         db_type="DUCKDB")

In [5]:
run_manager.rerun_cluster(run_name=run_name,
                          prompt = user_input,
                          category_name=CATEGORY,
                          num_clusters=NUM_CLUSTERS)

SELECT INSIGHTS_FILE_NAME,CONTAINER_NAME,ACCOUNT_NAME FROM run_log WHERE CATEGORY = ?             AND PARENT_CLUSTER_NAME IS NULL 
Connected to DuckDB database: itops.duckdb
Fetched 5 rows.
Connection closed.
[('ITSM_Data-RUN000777-ITSM-PARQUET2.parquet', 'itops', 'stgtxtsql'), ('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN00055.parquet', 'itops', 'stgtxtsql'), ('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN00055.parquet', 'itops', 'stgtxtsql'), ('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN00055.parquet', 'itops', 'stgtxtsql'), ('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN0006.parquet', 'itops', 'stgtxtsql')]
Completed CLUSTERING PART 1 1 ROW
Completed CLUSTERING PART 1 2 ROW
Completed CLUSTERING PART 1 3 ROW
Completed CLUSTERING PART 1 4 ROW
Completed CLUSTERING PART 1 5 ROW
Completed CLUSTERING PART 1 6 ROW
Completed CLUSTERING FINAL
Connected to DuckDB database: itops.duckdb
Query executed successfully.
Connection closed.
CLUSTER Names are ['Discord Installation and Performance Issues'
 'Technical Support, So

In [6]:
from itops.storage.azure_blob.parquet_helper import ParquetHelper

In [7]:
FILENAME = 'ITSM_Data-RUN000777-ITSM-PARQUET2-RUN0006.parquet'
CONTAINER_NAME = 'itops'
ACCOUNT_NAME = 'stgtxtsql'
blob_helper = AzureBlobHelper(account_name= ACCOUNT_NAME,
                              container_name= CONTAINER_NAME,
                              account_key=CONFIGS.AZURE_BLOB_STORAGE_KEY)
file_helper = ParquetHelper(azure_blob_helper=blob_helper)
df = file_helper.read_file(FILENAME)

In [8]:
df.columns

Index(['ID_request', 'Text', 'Category', 'Solution', 'Date_request_recieved',
       'Date_request_solved', 'ID_agent', 'themes', 'embeddings', 'CLUSTERS',
       'CLUSTER_NAMES'],
      dtype='object')

In [9]:
df["themes"].unique()

array(['Discord server management and moderation.',
       'Installing Discord on new PC.',
       'Discord disappeared after Windows update.',
       'Technical issue with Discord.',
       'Discord installation requirements.',
       'Discord freezing, update issue.',
       'Request for Discord emojis feature.', 'Installation needed.',
       'Discord notification and freezing issues.',
       'Urgent installation of Discord.',
       'IT Support, Prioritization, Frustration',
       'Discord group and channel management',
       'Importance of Discord access',
       'Discord installation on new laptop.',
       'Audio communication issues on Discord',
       '"Challenges of Space Exploration"',
       'Project management tool installation request',
       'Reinstallation of Asana after Windows update.',
       'Task completion issue in Asana', 'Asana not working, need help.',
       'Software update issue on PC.', 'Asana not working after restart.',
       'Error opening Asana app

In [10]:
len(df["CLUSTER_NAMES"].unique())

6

In [11]:
df["CLUSTERS"].unique()

array([5, 2, 0, 1, 4, 3], dtype=int32)

In [12]:
CLUSTER_NAMES=df["CLUSTER_NAMES"].unique()

In [13]:
CLUSTER_NAMES[0],len(CLUSTER_NAMES)

('Discord Installation and Performance Issues', 6)

In [14]:
for cluster_name in CLUSTER_NAMES:
    print(cluster_name)

Discord Installation and Performance Issues
Technical Support, Software Installation, Browser Issues
IT Support and Access Issues
Outlook Email Issues and Errors
CRM Technical Issues and Requests
Device replacement and ordering accessories


In [15]:
counts = df["CLUSTER_NAMES"].value_counts()

In [16]:
for item in counts.items():
    print(item[0],item[1])

Outlook Email Issues and Errors 28
Device replacement and ordering accessories 22
IT Support and Access Issues 20
Technical Support, Software Installation, Browser Issues 20
Discord Installation and Performance Issues 14
CRM Technical Issues and Requests 14


SUB CLUSTERING

In [17]:
PARENT_CLUSTER_NAME = 'CRM Technical Issues and Requests'
RUN_NAME = 'CRM0001'
PARENT_RUN_NAME = run_name

run_manager.rerun_sub_cluster(run_name=RUN_NAME,
                          prompt = user_input,
                          category_name=CATEGORY,
                          num_clusters=3,
                          parent_cluster_name =PARENT_CLUSTER_NAME,
                          parent_run_name = PARENT_RUN_NAME)

SELECT INSIGHTS_FILE_NAME,CONTAINER_NAME,ACCOUNT_NAME FROM run_log WHERE CATEGORY = ?             AND RUN_NAME = ? AND PARENT_CLUSTER_NAME IS NULL 
Connected to DuckDB database: itops.duckdb
Fetched 2 rows.
Connection closed.
[('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN0006.parquet', 'itops', 'stgtxtsql'), ('ITSM_Data-RUN000777-ITSM-PARQUET2-RUN0006.parquet', 'itops', 'stgtxtsql')]
Completed CLUSTERING PART 1 1 ROW
Completed CLUSTERING PART 1 2 ROW
Completed CLUSTERING PART 1 3 ROW
Completed CLUSTERING FINAL
Connected to DuckDB database: itops.duckdb
Query executed successfully.
Connection closed.
Connected to DuckDB database: itops.duckdb
CONNECTED to the database
Query executed successfully.
Data inserted successfully.
Connection closed.
Connected to DuckDB database: itops.duckdb
CONNECTED to the database
Query executed successfully.
Data inserted successfully.
Connection closed.
Connected to DuckDB database: itops.duckdb
CONNECTED to the database
Query executed successfully.
Data insert

In [18]:
FILENAME = 'ITSM_Data-RUN000777-ITSM-PARQUET2-RUN0006-CRM0001.parquet'
CONTAINER_NAME = 'itops'
ACCOUNT_NAME = 'stgtxtsql'
blob_helper = AzureBlobHelper(account_name= ACCOUNT_NAME,
                              container_name= CONTAINER_NAME,
                              account_key=CONFIGS.AZURE_BLOB_STORAGE_KEY)
file_helper = ParquetHelper(azure_blob_helper=blob_helper)
df = file_helper.read_file(FILENAME)

In [19]:
df["CLUSTER_NAMES"].value_counts()

CLUSTER_NAMES
Salesforce CRM installation request.    6
CRM Technical Issues                    5
Hubspot CRM technical issues            3
Name: count, dtype: int64